In [1]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\hp\\Files\\DataScience\\End-To-End Projects\\Chicken-Disease-Classification-Projects'

In [2]:

from dataclasses import dataclass 
from pathlib import Path

@dataclass 
class EvaluationConfig:
    training_data_path:Path
    trained_model_path:Path 
    all_params:dict 
    params_image_size:list 
    params_batch_size:int 
    
    


In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(self, 
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE, 
            params_batch_size = self.params.BATCH_SIZE,
            trained_model_path = Path(self.config.training.trained_model_path),
            training_data_path = Path(self.config.training.training_data_dir),
        )
        return eval_config



In [11]:
import tensorflow as tf
from cnnClassifier.utils.common import save_json
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config 

    def _valid_generator(self):
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1./255,
            validation_split = 0.20
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod 
    def load_model(path):
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.trained_model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss":self.score[0],
                  "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data=scores)

In [12]:
try:
    config = ConfigurationManager() 
    val_config = config.get_evaluation_config() 
    evaluation = Evaluation(val_config)
    evaluation.evaluation() 
    evaluation.save_score() 
except Exception as e:
    raise e

[2025-10-06 23:21:17,436] [INFO] [root:read_yaml:22] - reading yaml file -> config\config.yaml successed.
[2025-10-06 23:21:17,443] [INFO] [root:read_yaml:22] - reading yaml file -> params.yaml successed.
[2025-10-06 23:21:17,445] [INFO] [root:create_directories:42] - folder 'artifacts' already exist.
[2025-10-06 23:21:17,774] [WARNING] [absl:try_build_compiled_arguments:249] - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 78 images belonging to 2 classes.


c:\Users\hp\Files\DataScience\Anaconda\envs\cnn-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5000 - loss: 11.3637
